<a href="https://colab.research.google.com/github/sumaiya008/CCNY-DSE-Capstone-Project-Segmenting-Coral-Branch-tips/blob/main/notebooks/1.2_Model_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import random
import pickle
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load the augmented data from the saved file
data = np.load('/content/drive/My Drive/Capstone/Pickled/coral_augmented_combined_vgg16.npz')

# Access the arrays from the loaded data
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

# Check the shape of the loaded data
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (5104, 224, 224, 3)
y_train shape: (5104,)
X_test shape: (1276, 224, 224, 3)
y_test shape: (1276,)


In [3]:
# Assuming y_train and y_test contain labels in string format
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [4]:
# Load the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [8]:
# Add custom layers for your classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  # Reduced to 128 units
x = Dropout(0.5)(x)  # Add dropout layer with a rate of 0.5 (adjust as needed)
predictions = Dense(2, activation='softmax')(x)  # Adjust the number of output classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Define the optimizer (e.g., Stochastic Gradient Descent)
sgd = SGD(lr=0.001, momentum=0.9)

# Compile the model
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [9]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
160/160 [==============================] - 102s 492ms/step - loss: nan - accuracy: 0.5086 - val_loss: nan - val_accuracy: 0.5110
Epoch 2/10
 91/160 [================>.............] - ETA: 26s - loss: nan - accuracy: 0.5192

KeyboardInterrupt: ignored

In [5]:
# Import LearningRateScheduler from Keras callbacks
from keras.callbacks import LearningRateScheduler

# Define a learning rate schedule function
def learning_rate_schedule(epoch):
    if epoch < 5:
        return 0.01  # Initial learning rate for the first 5 epochs
    else:
        return 0.001  # Reduced learning rate for subsequent epochs

# Create a LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(learning_rate_schedule)

# Load the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for your classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  # Reduced to 128 units
x = Dropout(0.5)(x)  # Add dropout layer with a rate of 0.5 (adjust as needed)
predictions = Dense(2, activation='softmax')(x)  # Adjust the number of output classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Define the optimizer (e.g., Stochastic Gradient Descent)
sgd = SGD(lr=0.001, momentum=0.9)

# Compile the model
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with learning rate scheduling
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=16, callbacks=[lr_scheduler])


Epoch 1/10
319/319 [==============================] - 94s 240ms/step - loss: nan - accuracy: 0.5092 - val_loss: nan - val_accuracy: 0.5110 - lr: 0.0100
Epoch 2/10
319/319 [==============================] - 69s 215ms/step - loss: nan - accuracy: 0.5086 - val_loss: nan - val_accuracy: 0.5110 - lr: 0.0100
Epoch 3/10
319/319 [==============================] - 68s 215ms/step - loss: nan - accuracy: 0.5086 - val_loss: nan - val_accuracy: 0.5110 - lr: 0.0100
Epoch 4/10
319/319 [==============================] - 68s 214ms/step - loss: nan - accuracy: 0.5086 - val_loss: nan - val_accuracy: 0.5110 - lr: 0.0100
Epoch 5/10
109/319 [=========>....................] - ETA: 41s - loss: nan - accuracy: 0.5195

KeyboardInterrupt: ignored